In [21]:
get_mb_from_yymmdd(220823,'%Y-%m-%d %H:%M:%S')

'2022-08-23 00:00:00'

In [3]:
import datetime
import math
import time
import trino
import json
from trino.dbapi import connect
conn = connect(host="presto.crofarm.com",port=443,catalog='hive',http_scheme='https',auth=trino.auth.BasicAuthentication("db.manager","crofarm"))
cur = conn.cursor()


def get_mb_from_yymmdd(yymmdd, formt="%b %d, %a", days=0):
        t = time.strptime(str(yymmdd), "%y%m%d")
        newdate = datetime.date(t.tm_year, t.tm_mon,t.tm_mday) + datetime.timedelta(days)
        return newdate.strftime(formt)


def transpose_slot(slot_id):
        slot_id = str(slot_id)
        return slot_id[4:6] + slot_id[2:4] + slot_id[0:2]
        
    
def get_previous_working_date(ddmmyy, holiday_list, days=1):
        t = time.strptime(str(ddmmyy), '%d%m%y')
        newdate = datetime.date(t.tm_year, t.tm_mon,t.tm_mday) + datetime.timedelta(-days)
        while(newdate.strftime("%a").lower() in holiday_list):
            newdate = newdate + datetime.timedelta(1)
        return newdate.strftime("%d%m%y").lower()
    
    
def convert_into_db_format(epoch, t_format='%Y-%m-%d %H:%M:%S'):
        try:
            return datetime.datetime.fromtimestamp((int(epoch))).strftime(t_format)
        except Exception as e:
            pass
        
def get_logistics_cost_b2b(sales, n_slot_id):
#     total_routes = DtRouteHistory.objects.filter(slot_id=n_slot_id,is_agent=0,state=3).count()
    q="""Select count(1) from products.dt_route_history where slot_id={n_slot_id} and is_agent=FALSE and state=3 """.format(n_slot_id=n_slot_id)
    data=cur.execute(q)
    total_routes=0
   
    for d in data:
        total_routes=d[0]
        
    print(total_routes)
    if sales > 0:
        logistics_cost_per = int((total_routes*1400/float(sales))*100)
    else:
        logistics_cost_per = 0

    return total_routes*1400, logistics_cost_per


def get_b2b_stock_value(n_slot_id,distributor_id):
    query="select y.price, x.quantity from (select sum(hq_prediction) as quantity, prod_id from products.inventory_status where slot_id="+str(n_slot_id)+" and distributor_id="+str(distributor_id)+" and hq_prediction>0 group by prod_id)x join (select distinct pod.prod_id,pod.price from products.cr_purchase_order_data pod join products.cr_purchase_order po on pod.purchase_order_id = po.id where po.n_slot_id="+str(n_slot_id)+" and po.distributor_id="+str(distributor_id)+") y on x.prod_id=y.prod_id"
    data=cur.execute(query)
    total_b2b_stock_value=0
    for d in data:
        if d:
            total_b2b_stock_value +=float(d[0])*float(d[1])
    return round(total_b2b_stock_value,2)



def get_supplier_info(po_id_list):
    order_id_list=str(", ".join(str(id) for id in po_id_list))
    print(order_id_list)
    query = "select id,supplier_id from products.cr_purchase_order where id in ("+order_id_list+")"
    purchase_order=[]
    data=cur.execute(query)
    for d in data:
        data_dict={}
        data_dict['supplier_id']=d[1]
        purchase_order.append(data_dict)
        
    
    supplier_id_list = [p_order['supplier_id'] for p_order in purchase_order]
    supplier_data_list = get_supplier_data(supplier_id_list)
    for order in purchase_order:
        order["supplier_name"] = filter(lambda x: x['id'] == order["supplier_id"],supplier_data_list)[0]["name"]
        order["phone"] = filter(lambda x: x['id'] == order["supplier_id"],supplier_data_list)[0]["phone"]
    return purchase_order


def get_purchase_price_new(order_data, po_data_list):
    if Decimal(order_data['delivered_quantity']) > 0:
        try:
            purchase_data = json.loads(order_data["purchase_info"])
        except:
#             print "purchase_info not found"
            return 0, ""
        total_quantity = 0
        total_price = 0
        for pur_data in purchase_data:
            pd_dt = filter(lambda x: x['id'] == pur_data["order_data_id"], po_data_list)[0]
            if pd_dt['parent_id']:
                price = Decimal(pd_dt["shift_price"])
            else:
                price = Decimal(pd_dt["price"]) * (1+Decimal(pd_dt["per_inc"]))
            if not price:
                continue
            total_quantity += Decimal(pur_data['quantity'])
            total_price += Decimal(pur_data['quantity']) * Decimal(price)
        if total_price < 0.2 and total_quantity > 0.01:
            pur_data_obj="""select quantity ,price ,liq_price ,sell_quantity ,liquidate_quantity ,wastage_quantity ,prod_id ,prod_name ,subcat ,cast(concat(substr(cast(cast(from_unixtime(entry_time/1000) as timestamp) as varchar),1,19),cast('.' as varchar),substr(cast(entry_time as varchar),11,13)) as timestamp) ) as entry_time ,purchase_order_id ,unit ,inv_place ,is_farm ,image_id ,moq,po_price ,price_variance,sell_quantity ,return_quantity ,shrink_quantity ,status ,clear_quantity ,per_inc ,"source",in_kg ,cgst ,igst ,sgst ,discount_marked ,accepted_quantity ,parent_id ,shift_price ,mrp ,best_before ,expiry_date ,lot_number ,manufacturing_date ,cess_percentage ,cess_value ,is_cess ,tax ,tax_value ,dt from products.cr_purchase_order_data  where prod_id="""+prod_id['prod_id']+""" and id=<"""+pur_data["order_data_id"] +"""and price=0.2"""

        # print total_price, pur_data,order_data['prod_id']
            if pur_data_obj:
                if pur_data_obj[0].parent_id:
                    return pur_data_obj[0].shift_price, pur_data_obj[0].entry_time.strftime('%a, %d %b')
                return pur_data_obj[0].price*(1+pur_data_obj[0].per_inc), pur_data_obj[0].entry_time.strftime('%a, %d %b')
            else:
                pur_data_obj="""select quantity ,price ,liq_price ,sell_quantity ,liquidate_quantity ,wastage_quantity ,prod_id ,prod_name ,subcat ,cast(concat(substr(cast(cast(from_unixtime(entry_time/1000) as timestamp) as varchar),1,19),cast('.' as varchar),substr(cast(entry_time as varchar),11,13)) as timestamp) ) as entry_time ,purchase_order_id ,unit ,inv_place ,is_farm ,image_id ,moq,po_price ,price_variance,sell_quantity ,return_quantity ,shrink_quantity ,status ,clear_quantity ,per_inc ,"source",in_kg ,cgst ,igst ,sgst ,discount_marked ,accepted_quantity ,parent_id ,shift_price ,mrp ,best_before ,expiry_date ,lot_number ,manufacturing_date ,cess_percentage ,cess_value ,is_cess ,tax ,tax_value ,dt from products.cr_purchase_order_data  where prod_id="""+prod_id['prod_id']+""" and id=>"""+pur_data["order_data_id"] +"""and price=0.2"""
                if pur_data_obj:
                    if pur_data_obj[0].parent_id:
                        return pur_data_obj[0].shift_price, pur_data_obj[0].entry_time.strftime('%a, %d %b')
                    return pur_data_obj[0].price*(1+pur_data_obj[0].per_inc), pur_data_obj[0].entry_time.strftime('%a, %d %b')

        if total_quantity > 0:
            return total_price / total_quantity, ""
        else:
            return 0, ""
    else:
        return 0, ""
    
    
def get_purchase_price_of_pur(pur_data, po_data_list):
    if Decimal(pur_data['quantity']) > 0:
        po_data = filter(lambda x: x['id'] == pur_data["order_data_id"],po_data_list)[0]
        price = po_data["price"] * (1 + po_data['per_inc'])
        total_quantity = Decimal(pur_data['quantity'])
        total_price = Decimal(pur_data['quantity']) * Decimal(price)
        if total_price < 0.2 and total_quantity > 0.01:
            pur_data_obj="""select quantity ,price ,liq_price ,sell_quantity ,liquidate_quantity ,wastage_quantity ,prod_id ,prod_name ,subcat ,cast(concat(substr(cast(cast(from_unixtime(entry_time/1000) as timestamp) as varchar),1,19),cast('.' as varchar),substr(cast(entry_time as varchar),11,13)) as timestamp) ) as entry_time ,purchase_order_id ,unit ,inv_place ,is_farm ,image_id ,moq,po_price ,price_variance,sell_quantity ,return_quantity ,shrink_quantity ,status ,clear_quantity ,per_inc ,"source",in_kg ,cgst ,igst ,sgst ,discount_marked ,accepted_quantity ,parent_id ,shift_price ,mrp ,best_before ,expiry_date ,lot_number ,manufacturing_date ,cess_percentage ,cess_value ,is_cess ,tax ,tax_value ,dt from products.cr_purchase_order_data  where prod_id="""+prod_id['prod_id']+""" and id=<"""+pur_data["order_data_id"] +"""and price=0.2"""
            # print total_price, pur_data,order_data['prod_id']
            if pur_data_obj:
                return pur_data_obj[0].price * (1 + pur_data_obj[0].per_inc), pur_data_obj[0].entry_time.strftime('%a, %d %b')
            else:
                pur_data_obj = PurchaseOrderData.objects.filter(prod_id=po_data['prod_id'], id__gt=pur_data["order_data_id"], price__gte=0.2)[:1]
                pur_data_obj="""select quantity ,price ,liq_price ,sell_quantity ,liquidate_quantity ,wastage_quantity ,prod_id ,prod_name ,subcat ,cast(concat(substr(cast(cast(from_unixtime(entry_time/1000) as timestamp) as varchar),1,19),cast('.' as varchar),substr(cast(entry_time as varchar),11,13)) as timestamp) ) as entry_time ,purchase_order_id ,unit ,inv_place ,is_farm ,image_id ,moq,po_price ,price_variance,sell_quantity ,return_quantity ,shrink_quantity ,status ,clear_quantity ,per_inc ,"source",in_kg ,cgst ,igst ,sgst ,discount_marked ,accepted_quantity ,parent_id ,shift_price ,mrp ,best_before ,expiry_date ,lot_number ,manufacturing_date ,cess_percentage ,cess_value ,is_cess ,tax ,tax_value ,dt from products.cr_purchase_order_data  where prod_id="""+prod_id['prod_id']+""" and id=>"""+pur_data["order_data_id"] +"""and price=0.2"""
                if pur_data_obj:
                    return pur_data_obj[0].price * (1 + pur_data_obj[0].per_inc), pur_data_obj[0].entry_time.strftime('%a, %d %b')

        if total_quantity > 0:
            return price, ""
        else:
            return 0, ""
    else:
        return 0, ""
    
    
    
def get_supplier_data(supplier_id_list):
    """
    Will provide the retailer

    """
    retailer_set = list(set(supplier_id_list))
    if not retailer_set:
        return []
    retailer_set = [str(sid) for sid in retailer_set]
    retailer_query = ",".join(retailer_set)
    query = "select id, name, phone from crofarmusers.cr_supplier where id in (" + retailer_query + ")"
    cur.execute(query)
    rows = cur.fetchall()
    retailer_data_list = []
    for row in rows:
        retailer_data_list.append({"id": row[0],
                                   "name": row[1],
                                   "phone": row[2]
                                   })
    return retailer_data_list


def get_order_data(slot_from, slot_to):
    orders= """select * from consumer.cn_order where n_slot_id between {slot_from} and {slot_to} and retailer_id in (856,4724) and status <> 'x' """.format(slot_from=slot_from,slot_to=slot_to)
    order_list = orders.values('id', 'retailer_id', 'slot_id', 'is_deal', 'entry_time', 'route_id', 'pushed_by','distributor_id')
    order_id_list = []
    retailer_id_list = []
    for order in order_list:
        order_id_list.append(order["id"])

    order_data ="Select * from consumer.cn_order_data where order_id in ({order_id_list})".format(order_id_list=order_id_list)
    order_data_list = order_data.values('id', "prod_id", 'transit_quantity', 'order_id', 'quantity', 'tent_price','delivered_quantity', 'price', 'prod_name', 'subcat', 'unit', 'purchase_info', 'in_kg')
    return  order_data_list


def get_b2corder_data(slot_from, slot_to):
    q1 ="select * from consumer.cn_order where n_slot_id>{slot_from} and n_slot_id<{slot_to} and status not in ('X')".format(slot_from=slot_from,slot_to=slot_to)
    orders=cur.execute(q1)
    order_list = orders.values('id','n_slot_id')
    order_id_list = []
    retailer_id_list = []
    for order in order_list:
        order_id_list.append(order["id"])
    order_data = """Select * from consumer.cn_order_data where order_id in ({order_id_list})""".format(order_id_list=order_id_list)
    order_data_list = order_data.values(
        "prod_id",  'order_id', 'quantity', 'transit_quantity',
         'price', 'prod_name', 'subcat', 'unit','in_kg')
    return order_data_list



def get_repeat_consumer(consumer_id, n_slot_id):
    query="""Select count(1) from consumer.cn_order where consumer_id={consumer_id} and n_slot_id<={n_slot_id} and status not in 'X'""".format(consumer_id=consumer_id,n_slot_id=n_slot_id)
    cur.execute(query)
    order_count = cur.fetchall()
    if order_count > 1:
        return 1
    else:
        return 0
    
    
# def get_all_consumers(n_slot_id):
#     total_count=0
#     end_slot_id = str(get_mb_from_yymmdd(n_slot_id,'%Y-%m-%d %H:%M:%S'))
#     consumers="""Select count(1) from consumer.cn_consumer where cast(concat(substr(cast(cast(from_unixtime(entry_time/1000) as timestamp) varchar),1,19),cast('.' as varchar),substr(cast(entry_time as varchar),11,13)) as timestamp)=cast({end_slot_id} as varchar) and agent_id not in(4034, 2969) """.format(end_slot_id=end_slot_id)
#     cur.execute(consumers)
#     data = cur.fetchall()
#     return data[0][0]

def get_all_resellers(n_slot_id):
    query = "select id from crofarmusers.cr_retailer where is_agent = 1 and cast(cast(concat(substr(cast(cast(from_unixtime(entry_time/1000) as timestamp) varchar),1,19),cast('.' as varchar),substr(cast(entry_time as varchar),11,13)) as timestamp)as date) <= '20"+str(n_slot_id)[:2]+"-"+str(n_slot_id)[2:4]+"-"+str(n_slot_id)[4:6]+"'"
    cur.execute(query)
    row_list = cur.fetchall()
    return len(row_list)


def get_active_consumer(n_slot_id):
    active_count = 0
    try:
        q1 = """Select count(distinct(consumer_id)) as consumer_id__count from consumer.cn_order where n_slot_id={n_slot_id} and status in('D','PU','P') and agent_id not in(4034, 2969)""".format(n_slot_id=n_slot_id)
        cur.execute(q1)
        consumers = cur.fetchall()
        consumers=consumers[0][0]
        q2="""Select count(distinct(consumer_id)) as consumer_id__count from consumer.cn_order where n_slot_id={n_slot_id} and status in('D','PU','P') and agent_id not in(4034, 2969)""".format(n_slot_id=n_slot_id)
        cur.execute(q2)
        agents = cur.fetchall()
        agents=agents[0][0]
    except:
        consumers, agents = 0, 0
    return consumers, agents




def get_monthly_active_consumer(n_slot_id):
    ddmmyy = transpose_slot(n_slot_id)
    prev_ddmmyy = get_previous_working_date(ddmmyy, [], 30)
    start_slot = transpose_slot(prev_ddmmyy)
    active_count = 0
    try:
        q1 = """Select count(distinct(consumer_id)) as consumer_id__count from consumer.cn_order where n_slot_id={n_slot_id} and status in('D','PU','P') and agent_id not in(4034, 2969)""".format(n_slot_id=n_slot_id)
        cur.execute(q1)
        consumers = cur.fetchall()
        consumers=consumers[0][0]
        q2="""Select count(distinct(consumer_id)) as consumer_id__count from consumer.cn_order where n_slot_id={n_slot_id} and status in('D','PU','P') and agent_id not in(4034, 2969)""".format(n_slot_id=n_slot_id)
        cur.execute(q2)
        agents = cur.fetchall()
        agents=agents[0][0]
    except:
        consumers, agents = 0,0
    return consumers, agents


def get_referral_cost(n_slot_id):
    query="""select * from consumer.cn_wallet_transaction where added_by in (800000,1200000) and n_slot_id={n_slot_id}""".format(n_slot_id=n_slot_id)
    cur.execute(query)
    wallet_transactions= cur.fetchall()
    referral=0
    referral = sum(int(x.credit) for x in wallet_transactions)
    debit = sum(int(x.debit) for x in wallet_transactions)
    return referral-debit

def get_wallet_cost(n_slot_id):
    query= """Select * from consumer.cn_wallet_transaction where coupon_code in ('get100','get50','super','savebig','santa', 'holi21','aprl21','sumr21')""".format(n_slot_id=n_slot_id)
    cur.execute(query)
    wallet_transactions= cur.fetchall()
    wallet_cost=0
    for tran in wallet_transactions:
        if tran.coupon_code:
            if tran.coupon_code.lower() == 'sumr21' or tran.coupon_code.lower() == 'aprl21':
                if tran.credit >= 1199 and tran.credit <= 2998:
                    wallet_cost += 51
                elif tran.credit >= 2999 and tran.credit <= 5098:
                    wallet_cost += 151
                elif tran.credit >= 5099:
                    wallet_cost += 251
                else:
                    wallet_cost += 0
    return wallet_cost

def get_refund_cost(n_slot_id):
    q1="""Select * from consumer.cn_wallet_transaction where added_by >=0 and n_slot_id={n_slot_id}not in added_by in (300000,600000, 700000, 800000, 1200000,300001)"""
    cur.execute(q1)
    wallet_transactions= cur.fetchall()
    refund = 0
    refund = sum(int(x.credit) for x in wallet_transactions)
    return refund


def get_nps(orders):
    five_stars, four_stars, lt_four_stars, awaited, four_point_five=get_feedback(orders)
    total_feedback = five_stars + four_stars+ four_point_five + lt_four_stars
    if total_feedback > 0:
        promoters_per = int((five_stars+four_point_five)*100/float(total_feedback))
        detractor_per = int(lt_four_stars*100/float(total_feedback))
    else:
        promoters_per = 0
        detractor_per = 0
    nps = promoters_per - detractor_per
    return nps






def get_shortfall_refund_cost(orders):
    order_id_list = [int(x.id) for x in orders]
    query="""select * from consumer.cn_order_rating  where added_by=100000 and order_id in({order_ids})""".format(order_ids=order_ids)
    cur.execute(query)
    wallet_transactions=cur.execute(query)
    refund = 0
    refund = sum(int(x.credit) for x in wallet_transactions)
    return refund



def get_margin(n_slot_id):
    marketing_cost = get_marketing_cost(n_slot_id)
    if not marketing_cost:
        marketing_cost = 0
    b2corder_data_list = get_b2corder_data(n_slot_id, n_slot_id)
    prod_id_list = [ord_data["prod_id"] for ord_data in b2corder_data_list]
    prod_id_list = list(set(prod_id_list))
    # Cr ORDER DATA LIST
    orders_data_list = get_order_data(n_slot_id, n_slot_id)
    po_id_list = set()
    po_data_id_list = []
    purchase_price_prod_id_dict = {}
    for od in orders_data_list:
        try:
            purchase_info = json.loads(od.get('purchase_info', []))
        except Exception:
                purchase_info = []
        for dt in purchase_info:
            po_id_list.add(dt['po_id'])
            po_data_id_list.append(dt['order_data_id'])

        #supplier_data = get_supplier_info(po_id_list)
    po_data_list="""Select prod_id,id,price,per_inc,shift_price,parent_id from products.cr_purchase_order_data where id in ({po_data_id_list})""".format(po_data_id_list=po_data_id_list) 
    for od_data in orders_data_list:
        purchase_price, supp_info = get_purchase_price_new(od_data, po_data_list)
        purchase_price_prod_id_dict.update({od_data["prod_id"]:purchase_price})
                   
    total_sales_b2c=0  
    total_b2c_margin = 0      
    for prod_id in prod_id_list:
        prod_total_amount = 0
        prod_total_margin = 0
        prod_b2c =  filter(lambda x:x["prod_id"]==prod_id, b2corder_data_list)
        prod_purchase_price =  purchase_price_prod_id_dict.get(prod_id, 0)
#         print prod_id, prod_purchase_price
        for p_b2c in prod_b2c:
            prod_total_amount += p_b2c["transit_quantity"]* p_b2c["price"]
            prod_total_margin += p_b2c["transit_quantity"]*(float(p_b2c["price"])- float(prod_purchase_price))
        total_sales_b2c += prod_total_amount
        total_b2c_margin += prod_total_margin
    margin_per = 0  
    if total_sales_b2c > 0:
        margin_per = round(((total_b2c_margin+marketing_cost)/total_sales_b2c)*100,1)
    else:
        margin_per = 0
    margin_dict = {}
    margin_dict.update({n_slot_id:{'margin_per':margin_per, 'margin':int(total_b2c_margin+marketing_cost)}})
    return margin_dict, marketing_cost


def get_agent_commission(n_slot_id):
#     all_incentive = IncentiveLegder.objects.using('consumer').filter(n_slot_id=n_slot_id).exclude(agent_id__in=[2069, 4034]).aggregate(Sum('incentive'))
    query1= """Select sum(incentive) from consumer.cn_inc_ledger where n_slot_id ={n_slot_id} and agent_id not in(2069, 4034) """.format(n_slot_id=n_slot_id)

    all_incentives=0
    all_incentive=cur.execute(query1)
    for wallet in all_incentive:
        all_incentives=(wallet[0])
#     deduction_incentive = IncentiveLegder.objects.using('consumer').filter(n_slot_id=n_slot_id, inc_type__in=['ddd','dmi','dqi']).exclude(agent_id__in=[2069, 4034]).aggregate(Sum('incentive'))
    query2="select sum(incentive) from consumer.cn_inc_ledger where n_slot_id={n_slot_id} and inc_type in('ddd','dmi','dqi') and agent_id not in(2069, 4034)".format(n_slot_id=n_slot_id)
    deduction_incentive=cur.execute(query2)
    incentive_sum=0
    for d in deduction_incentive:
        incentive_sum+=(d[0])
    
    total_inc=all_incentives-incentive_sum
    
        
#     if deduction_incentive:
#         try:
#             total_inc = all_incentive['incentive__sum'] - deduction_incentive['incentive__sum']
#         except:
#             total_inc = all_incentive['incentive__sum']
#     else:
#         total_inc = all_incentive['incentive__sum'] 
    return int(total_inc)


def get_marketing_cost(n_slot_id):
    query='''select sum(marketing_cost)from products.order_marketing_cost where order_id in(select id from consumer.cn_order where n_slot_id ='''+n_slot_id+''' and status in ('D','P','PU'))'''
    cur.execute(query)
    marketing_cost = cur.fetchall()
    return marketing_cost[0][0]

def get_orders(n_slot_id):
    query="select count(1) from consumer.cn_order where n_slot_id="+n_slot_id+" and status in ('D','P','PU')"

def get_weekly_active_consumer(n_slot_id):
    ddmmyy = transpose_slot(n_slot_id)
    prev_ddmmyy = get_previous_working_date(ddmmyy, [], 6)
    start_slot = transpose_slot(prev_ddmmyy)
    active_count = 0
    try:
        q1 = """Select count(distinct(consumer_id)) as consumer_id__count from consumer.cn_order where n_slot_id={n_slot_id} and status in('D','PU','P') and agent_id not in(4034, 2969)""".format(n_slot_id=n_slot_id)
        cur.execute(q1)
        consumers = cur.fetchall()
        consumers=consumers[0][0]
        q2="""Select count(distinct(consumer_id)) as consumer_id__count from consumer.cn_order where n_slot_id={n_slot_id} and status in('D','PU','P') and agent_id not in(4034, 2969)""".format(n_slot_id=n_slot_id)
        cur.execute(q2)
        agents = cur.fetchall()
        agents=agents[0][0]
    except:
        consumers, agents = 0, 0
    return consumers, agents

def get_refund_cost(n_slot_id):
    q1="""Select credit from consumer.cn_wallet_transaction where added_by >=0 and n_slot_id={n_slot_id} and added_by not in (300000,600000, 700000, 800000, 1200000,300001)""".format(n_slot_id=n_slot_id)
    cur.execute(q1)
    wallet_transactions= cur.fetchall()
    refund = 0
    refund = sum(float(x[0]) for x in wallet_transactions)
    return refund

def get_sales(orders):
    sales=0
    for data in orders:
        sales+=float(data['total_amount'])+float(data['discount_amount'])
    return sales

def get_all_orders(n_slot_id):
    query="""select id,total_amount,agent_id,referral_agent_id,coupon_code,packaging_charge,discount_amount,consumer_id,item_count,star_rating from consumer.cn_order where n_slot_id="""+n_slot_id+""" and status in ('D','P','PU')"""
    cur.execute(query)
    data = cur.fetchall()
    order_list=[]
    for d in data:
        order_dict={}
        order_dict['id']=d[0]
        order_dict['total_amount']=d[1]
        order_dict['agent_id']=d[2]
        order_dict['referral_agent_id']=d[3]
        order_dict['coupon_code']=d[4]
        order_dict['packaging_charge']=d[5]
        order_dict['discount_amount']=d[6]
        order_dict['consumer_id']=d[7]
        order_dict['item_count']=d[8]
        order_dict['star_rating']=d[9]
        
        order_list.append(order_dict)
    return order_list



def get_pre_delivery_ratings(orders):
#     order_ids = [int(x['id']) for x in orders]
#     query="""select * from consumer.cn_order_rating where order_id in ({order_ids})""".format(order_ids=order_ids)
#     cur.execute(query)
#     order_ratings= cur.fetchall()
    five_stars, four_stars, lt_four_stars, awaited, four_point_five = 0, 0, 0, 0, 0
    for x in orders:
        if float(x['star_rating']) == float(5):
            five_stars += 1
        elif float(x['star_rating']) == float(4.5):
            four_point_five += 1
        elif float(x['star_rating']) == float(4):
            four_stars += 1
        elif float(x['star_rating']) > 0 and float(x['star_rating']) < float(4):
            lt_four_stars += 1
        else:
            awaited += 1
    return five_stars,four_stars,lt_four_stars,awaited,four_point_five

def get_coupon_cost(order_list):
    coupon_cost=0
    for data in order_list:
        coupon_cost+=float(data['discount_amount'])

    return coupon_cost

def get_cashback_coupon_cost(n_slot_id):
    query="""select sum(credit) from consumer.cn_wallet_transaction where n_slot_id="""+n_slot_id+""" and added_by=300001"""
    cur.execute(query)
    data = cur.fetchall()
    cashback_cost=0
    for d in data:
        cashback_cost=d[0] if d[0] else 0
    return cashback_cost

def get_wallet_cost(n_slot_id):
    query="""select credit,coupon_code from consumer.cn_wallet_transaction where coupon_code in ('get100','get50','super','savebig','santa', 'holi21','aprl21','sumr21') and n_slot_id="""+n_slot_id+""""""
    cur.execute(query)
    data = cur.fetchall()
    wallet_cost=0
    for tran in data:
        coupon_code=d[1]
        credit=d[0]
        if coupon_code and credit:
            if coupon_code.lower() == 'sumr21' or coupon_code.lower() == 'aprl21':
                if credit >= 1199 and credit <= 2998:
                    wallet_cost += 51
                elif credit >= 2999 and credit <= 5098:
                    wallet_cost += 151
                elif tran.credit >= 5099:
                    wallet_cost += 251
                else:
                    wallet_cost += 0
    return wallet_cost

def get_referral_cost(n_slot_id):
    query="select credit,debit from consumer.cn_wallet_transaction where added_by in (800000,1200000) and n_slot_id="+str(n_slot_id)+""
    cur.execute(query)
    data = cur.fetchall()
    referral_cost=0
    total_debit=0
    for d in data:
        credit=d[0] if d[0] else 0
        debit=d[1]  if d[1] else 0
        referral_cost+=float(credit)
        total_debit+=float(debit)

    return referral_cost-total_debit


def get_agent_commission(n_slot_id):
#     all_incentive = IncentiveLegder.objects.using('consumer').filter(n_slot_id=n_slot_id).exclude(agent_id__in=[2069, 4034]).aggregate(Sum('incentive'))
    query1= """Select sum(incentive) from consumer.cn_inc_ledger where n_slot_id ={n_slot_id} and agent_id not in(2069, 4034) """.format(n_slot_id=n_slot_id)
    cur.execute(query1)
    all_incentive=cur.fetchall()
    all_incentive=float(all_incentive[0][0])
#     deduction_incentive = IncentiveLegder.objects.using('consumer').filter(n_slot_id=n_slot_id, inc_type__in=['ddd','dmi','dqi']).exclude(agent_id__in=[2069, 4034]).aggregate(Sum('incentive'))
    query2="select sum(incentive) from consumer.cn_inc_ledger where n_slot_id={n_slot_id} and inc_type in('ddd','dmi','dqi') and agent_id not in(2069, 4034)".format(n_slot_id=n_slot_id)
    cur.execute(query2)
    deduction_incentive=cur.fetchall()
    deduction_incentive=(deduction_incentive[0][0])
    if deduction_incentive:
        try:
            total_inc = all_incentive - deduction_incentive
        except:
            total_inc = all_incentive
    else:
        total_inc = all_incentive
    return int(total_inc)


def get_new_consumers(orders,n_slot_id):
    new_consumers=0
    consumer_ids=[d['consumer_id'] for d in orders]
    if consumer_ids:
        consumer_ids=str(", ".join(str(id) for id in consumer_ids))

        query=" select count(distinct(consumer_id)) from consumer.cn_order where n_slot_id="+n_slot_id+" and consumer_id not in (select consumer_id from consumer.cn_order where n_slot_id<"+n_slot_id+" and status in ('D','P','PU')) and status in ('D','P','PU') "
        cur.execute(query)
        data = cur.fetchall()
        for d in data:
            new_consumers=d[0] if d[0] else 0

    return new_consumers

def get_new_agents(orders,n_slot_id):
    new_agents=0
    new_agent_ids=[d['agent_id'] for d in orders]

    if new_agent_ids:
        agent_ids=str(", ".join(str(id) for id in new_agent_ids))
        query="select count(distinct(agent_id)) from consumer.cn_order where n_slot_id="+n_slot_id+" and agent_id not in (select agent_id from consumer.cn_order where n_slot_id<"+n_slot_id+" and status in ('D','P','PU')) and status in ('D','P','PU')"
        cur.execute(query)
        data = cur.fetchall()
        for d in data:
            new_agents=d[0] if d[0] else 0

    return new_agents


def get_shortfall_refund_cost(orders):
    refund_cost=0
    order_id_list=[d['id'] for d in orders]
    if order_id_list:
        order_id_list=str(", ".join(str(id) for id in order_id_list))

        query="select sum(credit) from consumer.cn_wallet_transaction where added_by=100000 and order_id in ("+order_id_list+")"
        cur.execute(query)
        data = cur.fetchall()
        for d in data:
            refund_cost=d[0] if d[0] else 0


    return refund_cost

def get_sales(orders):
    sales=0
    for data in orders:
        sales+=float(data['total_amount'])+float(data['discount_amount'])
    return sales


def get_free_gifts(orders,n_slot_id):
    free_gifts=0
    order_id_list=[d['id'] for d in orders]

    if order_id_list:
        order_id_list=str(", ".join(str(id) for id in order_id_list))
        query="select count(1) from consumer.cn_order_data where order_id in ("+order_id_list+") and price=0 and prod_id not in (889)"
        cur.execute(query)
        data = cur.fetchall()
        for d in data:
            free_gifts=d[0] if d[0] else 0
            free_gifts=free_gifts*17

    return free_gifts


def get_issue_weight(orders):
    issue_weight=0
    order_id_list=[d['id'] for d in orders]
    if order_id_list:
        order_id_list=str(", ".join(str(id) for id in order_id_list))
        query="""select sum(credit) from consumer.cn_wallet_transaction where order_id in ({order_id_list}) and added_by=500000""".format(order_id_list=order_id_list)
        cur.execute(query)
        data = cur.fetchall()
        for d in data:
            issue_weight=d[0] if d[0] else 0


    return issue_weight



def get_packaging_charges(orders):
    packagin_charges=0
    for data in orders:
        packagin_charges+=float(data['packaging_charge'])
    return packagin_charges

def get_mkt_agents(orders):
    no_of_mkt_agents = 0
    mkt_agents=[]
    del_agents=[]
    agent = []
    for x in orders:
        agent.append(x['agent_id'])
        if x['referral_agent_id'] > 0 and int(x['agent_id']) != int(x['agent_id']):
            mkt_agents.append(x['referral_agent_id'])
            del_agents.append(x['agent_id'])
        else:
            del_agents.append(x['agent_id'])
    return len(list(set(mkt_agents))), len(list(set(del_agents)))


def get_marketing_and_delivery_orders(orders):
    no_of_mkt_agents = 0
    mkt_agents=[]
    del_agents=[]
    agent = []
    for x in orders:
        agent.append(x['agent_id'])
        if x['agent_id'] > 0 and int(x['referral_agent_id']) != int(x['agent_id']):
            mkt_agents.append(x['referral_agent_id'])
            del_agents.append(x['agent_id'])
        else:
            del_agents.append(x['agent_id'])

    return len(list(del_agents)), len(list(mkt_agents))


def get_all_consumers(n_slot_id):
    total_consumers=0
    end_slot_id=str(get_mb_from_yymmdd(n_slot_id,'%Y-%m-%d %H:%M:%S'))
    query="""select count(1) from consumer.cn_consumer where agent_id not in(4034, 2969) and cast(concat(substr(cast(cast(from_unixtime(entry_time/1000) as timestamp) as varchar),1,19),cast('.' as varchar),substr(cast(entry_time as varchar),11,13)) as timestamp)<=cast('{end_slot_id}' as timestamp)""".format(end_slot_id=end_slot_id)
    cur.execute(query)
    data = cur.fetchall()
    for d in data:
        total_consumers=d[0]
    return total_consumers


def get_active_consumer(n_slot_id):
    active_consumers=0
    active_resellers=0
    query=""" select count(distinct(consumer_id)) from  consumer.cn_order where status in ('D','P','PU') and agent_id not in (4034, 2969) and n_slot_id<={n_slot_id}""".format(n_slot_id=n_slot_id)
    cur.execute(query)
    data = cur.fetchall()
    for d in data:
        active_consumers=d[0] if d[0] else 0
    query2=""" select count(distinct(agent_id)) from  consumer.cn_order where status in ('D','P','PU') and agent_id not in (4034, 2969) and n_slot_id<={n_slot_id}""".format(n_slot_id=n_slot_id)
    cur.execute(query2)
    data = cur.fetchall()
    for d in data:
        active_resellers=d[0] if d[0] else 0
    return active_consumers,active_resellers

def get_all_resellers(n_slot_id):
    query = "select id from crofarmusers.cr_retailer where cast(date(cast(concat(substr(cast(cast(from_unixtime(entry_time/1000) as timestamp) as varchar),1,19),cast('.' as varchar),substr(cast(entry_time as varchar),11,13)) as timestamp) ) as varchar) <= '20"+str(n_slot_id)[:2]+"-"+str(n_slot_id)[2:4]+"-"+str(n_slot_id)[4:6]+"'"
    cur.execute(query)
    row_list = cur.fetchall()
    return len(row_list)

def get_weekly_active_consumer(n_slot_id):
    ddmmyy = transpose_slot(n_slot_id)
    prev_ddmmyy = get_previous_working_date(ddmmyy, [], 6)
    start_slot = transpose_slot(prev_ddmmyy)
    weekly_active_consumers=0
    weekly_active_resellers=0

    query=""" select count(distinct(consumer_id)) from  consumer.cn_order where status in ('D','P','PU') and agent_id not in (4034, 2969) and n_slot_id>={start_slot} and n_slot_id<={n_slot_id}""".format(start_slot=start_slot,n_slot_id=n_slot_id)
    cur.execute(query)
    data = cur.fetchall()
    for d in data:
        weekly_active_consumers=d[0] if d[0] else 0


    query=""" select count(distinct(agent_id)) from  consumer.cn_order where status in ('D','P','PU') and agent_id not in (4034, 2969) and n_slot_id>={start_slot} and n_slot_id<={n_slot_id}""".format(start_slot=start_slot,n_slot_id=n_slot_id)
    cur.execute(query)
    data = cur.fetchall()
    for d in data:
        weekly_active_resellers=d[0] if d[0] else 0

    return weekly_active_consumers,weekly_active_resellers

def get_monthly_active_consumer(n_slot_id):
    ddmmyy = transpose_slot(n_slot_id)
    prev_ddmmyy = get_previous_working_date(ddmmyy, [], 30)
    start_slot = transpose_slot(prev_ddmmyy)
    monthly_active_consumers=0
    monthly_active_resellers=0

    query=""" select count(distinct(consumer_id)) from  consumer.cn_order where status in ('D','P','PU') and agent_id not in (4034, 2969) and n_slot_id>={start_slot} and n_slot_id<={n_slot_id}""".format(start_slot=start_slot,n_slot_id=n_slot_id)
    cur.execute(query)
    data = cur.fetchall()
    for d in data:
        monthly_active_consumers=d[0] if d[0] else 0
    query=""" select count(distinct(agent_id)) from  consumer.cn_order where status in ('D','P','PU') and agent_id not in (4034, 2969) and n_slot_id>={start_slot} and n_slot_id<={n_slot_id}""".format(start_slot=start_slot,n_slot_id=n_slot_id)
    cur.execute(query)
    data = cur.fetchall()
    for d in data:
        monthly_active_resellers=d[0] if d[0] else 0
    return monthly_active_consumers,monthly_active_resellers



def get_welcome_bonus(n_slot_id):
    welcome_bonus=0
    query="""select sum(credit) from consumer.cn_wallet_transaction where n_slot_id={n_slot_id} and added_by in (600000, 700000)""".format(n_slot_id=n_slot_id)
    cur.execute(query)
    data = cur.fetchall()
    for d in data:
        welcome_bonus=d[0]

    return welcome_bonus


    
def get_welcome_used_bonus(n_slot_id):
    welcome_bonus=0
    query = """select dt1,sum(case when cnt1 = 1 then c_amt end) as used_bonus from (select K.dt1,K.id,K.c_amt,N.fdt,case when N.fdt>0 then 1 else 0 end as cnt1 from (select distinct consumer_id as id,n_slot_id as dt1,coalesce(credit,0) as c_amt from consumer.cn_wallet_transaction where n_slot_id = "+str(n_slot_id)+" and credit > 0 and added_by = 600000)K left join  (select id1,fdt from (select consumer_id as id1,min(n_slot_id) as fdt from consumer.cn_order co group by consumer_id)Q)N on K.id = N.id1)Y group by dt1"""
    cur.execute(query)
    data = cur.fetchall()
    for d in data:
        welcome_bonus=d[1]

    return welcome_bonus

def get_scheme_bonus(n_slot_id):
    scheme_bonus=0
    query="""select sum(credit) from consumer.cn_wallet_transaction where n_slot_id={n_slot_id} and added_by=300000""".format(n_slot_id=n_slot_id)
    cur.execute(query)
    data = cur.fetchall()
    for d in data:
        scheme_bonus=d[0]

    return scheme_bonus

def get_shortfall_items(n_slot_id):
    order_id_list=[]
    query=""" select count(1) from consumer.cn_order_data  where transit_quantity=0 and order_id in (select id from consumer.cn_order where n_slot_id={n_slot_id})""".format(n_slot_id=n_slot_id)
    cur.execute(query)
    row_list = cur.fetchall()
    return row_list[0][0]

def get_welcome_used_bonus(n_slot_id):
    welcome_bonus=0
    query = """select dt1,sum(case when cnt1 = 1 then c_amt end) as used_bonus from (select K.dt1,K.id,K.c_amt,N.fdt,case when N.fdt>0 then 1 else 0 end as cnt1 from (select distinct consumer_id as id,n_slot_id as dt1,coalesce(credit,0) as c_amt from consumer.cn_wallet_transaction where n_slot_id = """+(n_slot_id)+""" and credit > 0 and added_by = 600000)K left join  (select id1,fdt from (select consumer_id as id1,min(n_slot_id) as fdt from consumer.cn_order co group by consumer_id)Q)N on K.id = N.id1)Y group by dt1"""
    cur.execute(query)
    data = cur.fetchall()
    for d in data:
        welcome_bonus=d[1]

    return welcome_bonus

def get_order_complaints(n_slot_id, comp_type):
    query="select issue_id,count(1) from consumer.cn_issue_item where ticket_id in (select id from consumer.cn_ticket where n_slot_id={n_slot_id} and issue_type='item') group by issue_id having issue_id={comp_type}".format(n_slot_id=n_slot_id,comp_type=comp_type)
    cur.execute(query)
    row_list = cur.fetchall()
    if len(row_list)==0:
        return 0
    else:
        return row_list[0][1]


def get_delivery_by_10am_per(n_slot_id):
    query="Select actual_delivery_time from consumer.cn_order where n_slot_id={n_slot_id} and status!='X'".format(n_slot_id=n_slot_id)
    cur.execute(query)
    orders = cur.fetchall()
    on_time = 0
    for order in orders:
        if order[0]:
            delivery_time=datetime.datetime.fromtimestamp((int(order[0]))).strftime('%H:%M:%S')
            hours = int(delivery_time.split(":")[0])
            minutes = int(delivery_time.split(":")[1])
            if hours < 10:
                on_time += 1
            if hours == 10 and minutes <= 5:
                on_time += 1
    if len(orders):
        on_time_per = int((on_time/float(len(orders)))*100)
    else:
        on_time_per = 0
    return on_time_per


def get_gross_margin(n_slot_id):
#     from commodity.management.commands.crofarm_metrics import get_b2c_margin
    query = "select sum(gross_margin), sum(gross_margin_per), sum(sale_margin_val), sum(sec_sale_purchase_val), sum(sec_sale_val), sum(stock_transfer_margin_val) from products.dc_metric where slot_id = "+str(n_slot_id)
    cur.execute(query)
    rows = cur.fetchall()
    try:
        sec_sale_purchase = int(rows[0][3])
        sec_sale = int(rows[0][4])
    except:
        sec_sale_purchase, sec_sale = 0,0
    loss_on_sec_sale = sec_sale_purchase - sec_sale
    try:
        b2c_margin = get_b2c_margin(n_slot_id)
    except:
        b2c_margin = 0
    try:
        gross_margin = int(float(rows[0][2])) + int(float(rows[0][5])) - b2c_margin
        sales_margin = int(float(rows[0][2]))
        stm = int(float(rows[0][5]))
    except:
        sales_margin = 0
        stm = 0
        gross_margin = -b2c_margin

    return {"gross_margin":gross_margin,"sec_sale_purchase":\
    sec_sale_purchase, "sec_sale":sec_sale, "loss_on_sec_sale":loss_on_sec_sale, "sales_margin":sales_margin, "stock_transfer_margin":stm}

def get_b2b_metrics(n_slot_id):
    b2b_metrics={}
    q1="""Select * from consumer.cn_order where n_slot_id={n_slot_id}  and status in('D','M','C') and agent_id!=856 limit 1""".format(n_slot_id=n_slot_id)
    cur.execute(q1)
    orders = cur.fetchall()
    b2b_metrics.update({'no_of_retailers':len(orders)})
    revenue = sum([float(x[6]) for x in orders])
    b2b_metrics.update({'b2b_revenue':revenue})
    margin = get_gross_margin(n_slot_id)
    b2b_metrics.update({'margin_b2b':margin['gross_margin']
        })
    logistics_b2b, logistics_b2b_per = get_logistics_cost_b2b(revenue, n_slot_id)
    b2b_metrics.update({'logistics_b2b':logistics_b2b, 'logistics_b2b_per':logistics_b2b_per})
    net_margin_b2b = margin['gross_margin'] - logistics_b2b
    if revenue > 0:
        net_margin_b2b_per =  int((net_margin_b2b/float(revenue))*100)
    else:
        net_margin_b2b_per = 0
    b2b_metrics.update({'net_margin_b2b':net_margin_b2b, 'net_margin_b2b_per':net_margin_b2b_per})
    available_stock = get_b2b_stock_value(n_slot_id,6)
    stock_sold = revenue
    if revenue > 0:
        stock_sold_per =  int((revenue/float(available_stock))*100)
    else:
        stock_sold_per = 0
    b2b_metrics.update({'available_stock':available_stock, 'stock_sold':stock_sold, 'stock_sold_per':stock_sold_per})
    return b2b_metrics

def get_consumer_wallet_info(n_slot_id):
#     wallet_balance = WalletBalanceHistory.objects.using('consumer').filter(n_slot_id = n_slot_id).first()
    q1="""select * from consumer.wallet_balance_history where n_slot_id ={n_slot_id} limit 1""".format(n_slot_id=n_slot_id)
    cur.execute(q1)
    wallet_balance=cur.fetchall()
    if wallet_balance:
        return wallet_balance[0][2], wallet_balance[0][3], wallet_balance[0][4]
    else:
        return 0,0,0
    
def get_feedback(orders):
    five_stars, four_stars, lt_four_stars, awaited, four_point_five = 0, 0, 0, 0, 0
    for x in orders:
        if float(x['star_rating']) == 5:
            five_stars += 1
        elif float(x['star_rating']) == 4.5:
            four_point_five += 1
        elif float(x['star_rating']) == 4:
            four_stars += 1
        elif float(x['star_rating']) > 0 and float(x['star_rating']) < 4:
            lt_four_stars += 1
        else:
            awaited += 1
    return five_stars,four_stars,lt_four_stars,awaited,four_point_five

def get_nps(orders):
    five_stars, four_stars, lt_four_stars, awaited, four_point_five=get_feedback(orders)
    total_feedback = five_stars + four_stars+ four_point_five + lt_four_stars
    if total_feedback > 0:
        promoters_per = int((five_stars+four_point_five)*100/float(total_feedback))
        detractor_per = int(lt_four_stars*100/float(total_feedback))
    else:
        promoters_per = 0
        detractor_per = 0
    nps = promoters_per - detractor_per
    return nps



def get_pre_delivery_nps(orders):
    five_stars, four_stars, lt_four_stars, awaited, four_point_five=get_pre_delivery_ratings(orders)
    total_feedback = five_stars + four_stars+ four_point_five + lt_four_stars
    if total_feedback > 0:
        promoters_per = int((five_stars+four_point_five)*100/float(total_feedback))
        detractor_per = int(lt_four_stars*100/float(total_feedback))
    else:
        promoters_per = 0
        detractor_per = 0
    nps = promoters_per - detractor_per
    return nps



# def get_issue_weight(orders):
#     order_id_list = [int(x['id']) for x in orders]
# #     wallet_transactions = WalletTransaction.objects.using('consumer').filter(added_by=500000, order_id__in=order_id_list)
#     query="""select credit from consumer.cn_order_rating  where added_by=500000 and order_id in({order_ids})""".format(order_ids=order_id_list)
#     cur.execute(query)
#     wallet_transactions=cur.fetchall()
#     refund = 0
#     refund = sum(int(x[0]) for x in wallet_transactions)
#     return refund


def get_gateway_charges(n_slot_id):
    paytm_charges = {'CC':0.0235, 'DC':0.0202, 'NB':0.0210, 'UPI':0.0101, 'PPI':0.0199,'':0.02, 'None':0.02}
    razorpay_charges = {'card':0.01, 'netbanking1':0.0160, 'netbanking2':0.0130, 'upi':0.001, 'wallet':0.0175,'':0.015,'None':0.015}
#     paytm_transactions = CnPaytmTransaction.objects.using('consumer').filter(status='S', n_slot_id=n_slot_id)
    q1="""select *  from consumer.cn_paytm_transaction where status='S' and n_slot_id ={n_slot_id}""".format(n_slot_id=n_slot_id)
    cur.execute(q1)
    paytm_transactions=cur.fetchall()
    gateway_charges = 0
    paytm_amount = 0
    for paytm_trans in paytm_transactions:
        try:
            gateway_charges += float(paytm_trans[3]) * float(paytm_charges[paytm_trans[12]])
            paytm_amount += float(paytm_trans[3])
        except:
#             print ("not found paytm")
            gateway_charges += float(paytm_trans[3]) * float(0.02)
            paytm_amount += float(paytm_trans[3])


#     razorpay_transactions = CnRazorPayTransaction.objects.using('consumer').filter(status='S', n_slot_id=n_slot_id)
    q2="""Select * from consumer.cn_razorpay_transaction where status='S' and n_slot_id={n_slot_id}""".format(n_slot_id=n_slot_id)
    cur.execute(q1)
    razorpay_transactions=cur.fetchall()
    rz_amount=0
    for trans in razorpay_transactions:
        rz_amount += float(trans[3])
        try:
            if trans[5] != 'netbanking':
                gateway_charges += float(trans[3]) * float(razorpay_charges[trans[5]])
            elif trans[5] == 'netbanking':
                if trans.bank_name.lower() in ('hdfc','icici', 'axis','sbi','kotak'):
                    gateway_charges += float(trans[3]) * float(razorpay_charges['netbanking1'])
                else:
                    gateway_charges += float(trans[3]) * float(razorpay_charges['netbanking2'])
        except:
#             print ("not found")
            gateway_charges += float(trans[3]) * float(0.015)

    return int(float(gateway_charges))


def get_weight_refund(n_slot_id):
#     wallet_transactions2 = WalletTransaction.objects.using('consumer').filter(added_by=500000, n_slot_id=n_slot_id)
    q="""Select credit from consumer.cn_wallet_transaction where added_by =500000 and n_slot_id={n_slot_id}""".format(n_slot_id=n_slot_id)
    cur.execute(q)
    wallet_transactions2=cur.fetchall()
    weight_refund=0
    weight_refund = sum(float(x[0]) for x in wallet_transactions2)
    return weight_refund


def get_logistics_cost(sales, n_slot_id):
#     total_routes = DtRouteHistory.objects.filter(slot_id=n_slot_id,is_agent=1,state=3).count()
    q="""Select count(1) from products.dt_route_history where slot_id={n_slot_id} and is_agent=true and state=3""".format(n_slot_id=n_slot_id)
    cur.execute(q)
    total_routes=cur.fetchall()
    if sales > 0:
        logistics_cost_per = int(((total_routes[0][0])*1400/float(sales))*100)
    else:
        logistics_cost_per = 0

    return total_routes[0][0]*1400, logistics_cost_per


def get_wastage(today_slot_id):
    query_string = "select pd.prod_id, pd.prod_name, pd.subcat, (case when inv.quant_type in ('RI', 'RW') THEN inv.credit ELSE inv.debit END) as quantity, inv.unit,\
        (pd.price * (1 + pd.per_inc)) as po_price, ((case when inv.quant_type in ('RI', 'RW') THEN inv.credit ELSE inv.debit END) * (pd.price * (1 + pd.per_inc))) as po_value,\
        inv.price as sell_price, (inv.debit * inv.price) as sell_value,\
        concat(SUBSTRING(cast(inv.slot_id as varchar), 5, 2), '/', SUBSTRING(cast(inv.slot_id as varchar), 3, 2), '/', '20', SUBSTRING(cast(inv.slot_id as varchar), 1, 2)) Date,\
        concat(SUBSTRING(od.slot_id, 1, 2), '/', SUBSTRING(od.slot_id, 3, 2), '/', '20', SUBSTRING(od.slot_id, 5, 2)) PO_Date,\
        inv.po_id, rt.name as supplier, dt.name DC, inv.quant_type as type, inv.remark, inv.updated_by, dt.name DC\
        from \
        products.inventory_transaction as inv inner join products.cr_purchase_order as od on inv.po_id = od.id\
        and inv.slot_id <= " + str(today_slot_id) + " and inv.slot_id >= " + str(today_slot_id) + " and inv.quant_type in ('WA', 'RI', 'RW', 'SH', 'CL', 'DU', 'SU')\
        inner join products.cr_purchase_order_data as pd on pd.prod_id = inv.prod_id and pd.purchase_order_id=od.id\
        inner join crofarmUsers.cr_supplier as rt on rt.id = od.supplier_id\
        inner join crofarmUsers.cr_distributor as dt on dt.id = od.distributor_id and od.distributor_id in \
        (6,8,9)"
    
    row_list=cur.execute(query_string)
#     row_list=cur.fetchall()
#     print(row_list)
    dump_stock_list=[]
    prod_ids=set()
    
    for x in row_list:
#         print(x)
        dump_dict={}
        dump_dict['prod_id']=int(x[0])
        dump_dict['prod_name']=x[1]
        dump_dict['value']=float(x[6])-float(x[8])
        dump_dict['type']=str(x[14])
        dump_dict['dc']=str(x[13])
        dump_stock_list.append(dump_dict)
        prod_ids.add(int(x[0]))
        

    final_dump=[]
#     print(dump_stock_list)
    for prod_id in list(prod_ids):
        shrinkage = sum(x['value'] for x in filter(lambda x:x['prod_id']==prod_id and x['type'] == 'SH', dump_stock_list))
        trash = sum(x['value'] for x in filter(lambda x:x['prod_id']==prod_id and x['type'] == 'CL', dump_stock_list))
        loss = sum(x['value'] for x in filter(lambda x:x['prod_id']==prod_id and x['type'] == 'DU', dump_stock_list))
        wastage_value = sum(x['value'] for x in filter(lambda x:x['prod_id']==prod_id and x['type'] == 'WA', dump_stock_list))
        prod_name = filter(lambda x:x['prod_id']==prod_id, dump_stock_list)
#         for i in prod_name:
#             print(i['prod_name'])
#        [0]['prod_name']
        final_dump.append({"shrinkage":shrinkage, "trash":trash, "loss":loss, "total":shrinkage+wastage_value, "prod_id":prod_id, "prod_name":prod_name})
    final_dump = filter(lambda x:x['total']>0, final_dump)
    final_dump = sorted(final_dump, key=lambda x:x['total'], reverse=True)
    total_waste=0
    shrinkage=0
    trash=0
    for row in final_dump:
        total_waste += row['total']
        shrinkage += row['shrinkage']
        trash += row['trash']
    query = "select sum(net_amount) from consumer.cn_order where n_slot_id = "+str(today_slot_id)+" and status != 'X'"
    cur.execute(query)
    try:
        total_booked = cur.fetchall()[0][0]
#         print total_booked
        wastage_per = 0
        if int(total_booked) > 0:
            wastage_per = total_waste*100/Decimal(total_booked)
        else:
            wastage_per = 0
    except:
        wastage_per = 0
        pass
    return {"total_waste":int(total_waste),"shrinkage":int(shrinkage),"trash":int(trash), "wastage_per":round(wastage_per,2)}


def get_between_slot(self, start_slot, end_slot):
        slot_id_list = []
        by = datetime.date(
            int('20' + start_slot[4:6]), int(start_slot[2:4]),
            int(start_slot[:2]))
        end = datetime.date(
            int('20' + end_slot[4:6]), int(end_slot[2:4]), int(end_slot[:2]))
        slot_id_list.append(start_slot)
        for day in range(0, (end - by).days):
            time = by + datetime.timedelta(day + 1)
            slot_id_list.append(str(time.strftime('%d%m%y')))
        return slot_id_list

In [31]:
get_all_consumers('220823')

select count(1) from consumer.cn_consumer where agent_id not in(4034, 2969) and cast(concat(substr(cast(cast(from_unixtime(entry_time/1000) as timestamp) as varchar),1,19),cast('.' as varchar),substr(cast(entry_time as varchar),11,13)) as timestamp)<=cast('2022-08-23 00:00:00' as timestamp)


2136336

In [20]:
def get_consumer_order_info(n_slot_id):
    orders=get_all_orders(n_slot_id)
    print("length of orders",len(orders))
    coupon_cost=get_coupon_cost(orders)
    print("coupon_cost",coupon_cost)
    coupon_cost+=get_cashback_coupon_cost(n_slot_id)
    print("coupon_cost_with_cashback",coupon_cost)
    wallet_cost = get_wallet_cost(n_slot_id)
    print("get_wallet_cost",wallet_cost)
    referral_cost = get_referral_cost(n_slot_id)
    print("referral_cost",referral_cost)
    consumer_count, total_balance , avg_wallet =  get_consumer_wallet_info(n_slot_id)
    new_consumers = get_new_consumers(orders, n_slot_id)
    print("new_consumers",new_consumers)
    new_resellers=get_new_agents(orders,n_slot_id)
    print("new_agents",new_resellers)
    shortfall_refund = get_shortfall_refund_cost(orders)
    print("shortfall_refund",shortfall_refund)
    sales=get_sales(orders)
    print("sales",sales)
    booked_value = sales
    free_gifts = get_free_gifts(orders,n_slot_id)
    print("free_gifts",free_gifts)
    sales -= round(float(shortfall_refund))
    sales += round(float(free_gifts))
    five_stars,four_stars,lt_four_stars,awaited, four_point_five = get_feedback(orders)
    nps = get_nps(orders)
    pre_del_nps = get_pre_delivery_nps(orders)
    if sales > 0:
        shortfall_refund_per = round(((float(shortfall_refund)/float(sales))*100),1)
    else:
        shortfall_refund_per = 0
    
    issue_weight=get_issue_weight(orders) 
    weight_issue_by_500000=issue_weight
    print("issue_weight",issue_weight)
    sales -= round(float(issue_weight))
    packaging_charges=get_packaging_charges(orders)
    print("packagin_charges",packaging_charges)
    packaging_charges_after_gst = int(packaging_charges*100/float(118))
    sales -= packaging_charges - packaging_charges_after_gst
    booked_value -= packaging_charges
    sales_without_discount = sum(round(float(x['total_amount'])) for x in orders)
    no_of_agents = len(list(set([round(float(x['agent_id'])) for x in orders])))
    print("no_of_agents",no_of_agents)
    no_of_mkt_agents, no_of_del_agents = get_mkt_agents(orders)
    print("no_of_mkt_agents",no_of_mkt_agents)
    print("no_of_del_agents",no_of_del_agents)
    
    num_of_deli_and_mkt_orders, num_of_mkt_orders = get_marketing_and_delivery_orders(orders)
    print ("num_of_deli_and_mkt_orders ",num_of_deli_and_mkt_orders)
    print ("num_of_mkt_orders ",num_of_mkt_orders)

#     no_of_agents = no_of_mkt_agents + no_of_del_agents
    orders_count=len(orders)
    print("orders_count",orders_count)
    no_of_items = sum(int(x['item_count']) for x in orders)
    

    if orders_count > 0:x`x`
        avg_order_value = sales/orders_count
    else:
        avg_order_value = 0
    if no_of_agents > 0:
        avg_order_value_per_agent = (sales/no_of_agents)
    else:
        avg_order_value_per_agent = 0
    if no_of_items > 0:
        avg_item_value = int(sales/no_of_items)
    else:
        avg_item_value = 0
    if orders_count > 0:
        item_per_order = round(no_of_items/orders_count,2)
    else:
        item_per_order = 0

    if no_of_agents > 0:
        avg_order_per_agent = round(orders_count/no_of_agents,2)
    else:
        avg_order_per_agent = 0
    if orders_count > 0:
        five_stars_per = int((five_stars/float(orders_count))*100)
        four_stars_per = int((four_stars/float(orders_count))*100)
        lt_four_stars_per = int((lt_four_stars/float(orders_count))*100)
        awaited_per = int((awaited/float(orders_count))*100)
    else:
        five_stars_per = 0
        four_stars_per = 0
        lt_four_stars_per = 0
        awaited_per = 0
    total_consumers = get_all_consumers(n_slot_id)
    print("total_consumers",total_consumers)
    total_active_consumers, total_active_resellers = get_active_consumer(n_slot_id)
    print("total_consumers",total_active_consumers)
    print("total_consumers",total_active_resellers)
    total_resellers = get_all_resellers(n_slot_id)

    repeat_consumer = orders_count - new_consumers
    print("repeat_consumer",repeat_consumer)
    if orders_count > 0:
        repeat_consumer_per = int((repeat_consumer/orders_count)*100)
    else:
        repeat_consumer_per = 0

    repeat_reseller = no_of_agents - new_resellers
    print("repeat_reseller",repeat_reseller)
    if no_of_agents > 0:
        repeat_reseller_per = int((repeat_reseller/no_of_agents)*100)
    else:
        repeat_reseller_per = 0
    print("repeat_reseller_per",repeat_reseller_per)
    if total_consumers > 0:
        total_active_consumers_per = int((total_active_consumers/total_consumers)*100)
    else:
        total_active_consumers_per = 0

    if total_resellers > 0:
        total_active_resellers_per = int((total_active_resellers/total_resellers)*100)
    else:
        total_active_resellers_per = 0
    print("total_active_resellers_per",total_active_resellers_per)
    weekly_active_consumers, weekly_active_resellers= get_weekly_active_consumer(n_slot_id)
    print("weekly_active_consumers",weekly_active_consumers)
    print("weekly_active_resellers",weekly_active_resellers)
#     print "get_weekly_active_consumer ",time.time() - start_time 
    if total_consumers > 0:
        weekly_active_consumer_per = int((weekly_active_consumers/float(total_active_consumers))*100)
    else:
        weekly_active_consumer_per = 0
        
    if total_resellers > 0:
        weekly_active_reseller_per = int((weekly_active_resellers/float(total_active_resellers))*100)
    else:
        weekly_active_reseller_per = 0

    monthly_active_consumers, monthly_active_resellers = get_monthly_active_consumer(n_slot_id)
    print("monthly_active_consumers",monthly_active_consumers)
    print("monthly_active_resellers",monthly_active_resellers)
    
    shortfall_items = get_shortfall_items(n_slot_id)
    print("shortfall_items",shortfall_items)

    if total_consumers > 0:
        monthly_active_consumer_per = int((monthly_active_consumers/float(total_active_consumers))*100)
    else:
        monthly_active_consumer_per = 0

    if total_resellers > 0:
        monthly_active_reseller_per = int((monthly_active_resellers/float(total_active_resellers))*100)
    else:
        monthly_active_reseller_per = 0
    print("monthly_active_consumer_per",monthly_active_consumer_per)
    print("monthly_active_reseller_per",monthly_active_reseller_per)
    wallet_refund = get_refund_cost(n_slot_id)
    welcome_bonus = get_welcome_bonus(n_slot_id)
    print("welcome_bonus",welcome_bonus)
    scheme_bonus = get_scheme_bonus(n_slot_id)
    print("scheme_bonus",scheme_bonus)

    welcome_used_bonus = get_welcome_used_bonus(n_slot_id)
    print("welcome_bonus used",welcome_used_bonus)
    marketing_cost=get_marketing_cost(n_slot_id)
    print("marketing_cost",marketing_cost)
    agent_commission = get_agent_commission(n_slot_id)
    
    agent_commission = get_agent_commission(n_slot_id)
    total_cost = float(coupon_cost) + float(wallet_cost) + float(referral_cost) + float(welcome_used_bonus) + float(scheme_bonus) + float(free_gifts) + float(marketing_cost)
 
    wallet_refund=float(wallet_refund) - float(shortfall_refund)-float(weight_issue_by_500000) 
    net_refund = float(wallet_refund) - float(shortfall_refund)-float(weight_issue_by_500000) 
    if sales > 0:
        total_cost_per = int((total_cost/float(sales)*100))
    else:
        total_cost_per = 0
    
    if sales > 0:
        refund_per = round(((wallet_refund/float(sales))*100),1)
    else:
        refund_per = 0
    if sales_without_discount > 0:
        agent_commission_per = int((agent_commission/float(sales_without_discount))*100)
    else:
        agent_commission_per = 0
    if sales > 0:
        net_refund_per = round(((float(net_refund)/float(sales))*100),1)
    else:
        net_refund_per = 0
    if no_of_agents > 0:
        revenue_per_reseller = int(sales/no_of_agents)
    else:
        revenue_per_reseller = 0
    start_time = time.time()
    gateway_charges = get_gateway_charges(n_slot_id)
    weight_refund = get_weight_refund(n_slot_id)
    delivery_10am_per =  get_delivery_by_10am_per(n_slot_id)
    wastage = get_wastage(n_slot_id)
    logistics_cost, logistics_cost_per =  get_logistics_cost(sales, n_slot_id)
    dump_stock = wastage['total_waste'] - wastage['shrinkage']
    shrinkage = wastage['shrinkage']
    if sales > 0:
        gateway_charges_per = int((gateway_charges/float(sales))*100)
    else:
        gateway_charges_per = 0
    total_wastage = wastage['total_waste']
    if sales > 0:
        total_wastage_per= round((total_wastage/float(sales))*100,2)
    else:
        total_wastage_per = 0
    missing_items = get_order_complaints(n_slot_id, 5)
    quality_items = get_order_complaints(n_slot_id, 4)
    weight_items = get_order_complaints(n_slot_id, 2)
    wrong_items = get_order_complaints(n_slot_id, 3)
    broken_items = get_order_complaints(n_slot_id, 1)
    print ("get_delivery_by_10am_per ",time.time() - start_time)
    resp= {'orders_count':orders_count, 'no_of_items':no_of_items, 'marketing_cost':marketing_cost,'sales':sales,'no_of_agents':no_of_agents, 'delivery_marketing_order': num_of_deli_and_mkt_orders-num_of_mkt_orders, 'marketing_order':num_of_mkt_orders, 'no_of_mkt_agents': no_of_mkt_agents, 'no_of_del_agents':no_of_del_agents, 'avg_order_value':avg_order_value, 'avg_order_value_per_agent':avg_order_value_per_agent, 'avg_item_value':avg_item_value,'avg_order_per_agent':avg_order_per_agent, 'item_per_order':item_per_order, 'repeat_consumers':repeat_consumer, 'repeat_consumer_per':repeat_consumer_per, 'total_consumers':total_consumers,'total_resellers':total_resellers, 'weekly_active_consumers':weekly_active_consumers, 'weekly_active_resellers':weekly_active_resellers,'weekly_active_consumer_per':weekly_active_consumer_per, 'weekly_active_reseller_per':weekly_active_reseller_per,     'monthly_active_consumers':monthly_active_consumers,'monthly_active_resellers':monthly_active_resellers,'monthly_active_reseller_per':monthly_active_reseller_per,'monthly_active_consumer_per':monthly_active_consumer_per, 'referral_cost':referral_cost, 'coupon_cost':coupon_cost, 'wallet_cost':wallet_cost,'wallet_balance':total_balance, 'consumers':consumer_count, 'avg_wallet':avg_wallet, 'wallet_refund':wallet_refund, 'refund_per':refund_per, 'agent_commission':agent_commission, 'agent_commission_per':agent_commission_per, 'five_stars':five_stars, 'four_stars':four_stars,'lt_four_stars':lt_four_stars, 'awaited':awaited, 'total_cost':total_cost, 'total_cost_per':total_cost_per, 'five_stars_per':five_stars_per, 'four_stars_per':four_stars_per, 'awaited_per':awaited_per, 'lt_four_stars_per':lt_four_stars_per, 'active_consumers':total_active_consumers,'active_consumers_per':total_active_consumers_per,'total_active_resellers':total_active_resellers, 'total_active_resellers_per':total_active_resellers_per, 'shortfall_refund':shortfall_refund,'shortfall_refund_per':shortfall_refund_per, 'nps':nps,'pre_del_nps':pre_del_nps, 'net_refund':net_refund, 'net_refund_per':net_refund_per, 'gateway_charges':gateway_charges, 'new_consumers':new_consumers, 'blank':'    ', 'new_reseller':new_resellers, 'repeat_reseller':repeat_reseller, 'repeat_reseller_per':repeat_reseller_per, 'revenue_per_reseller':revenue_per_reseller, 'booked_value':booked_value+marketing_cost, 'weight_refund':weight_refund, 'dump_stock':dump_stock,'shrinkage':shrinkage,'total_wastage':total_wastage,'total_wastage_per':total_wastage_per,'logistics_cost':logistics_cost,'logistics_cost_per':logistics_cost_per, 'gateway_charges_per':gateway_charges_per,'welcome_bonus':welcome_bonus,'scheme_bonus':scheme_bonus, 'used_welcome_bonus': welcome_used_bonus, 'free_gift':0,'delivery_10am_per':delivery_10am_per,'missing_items':missing_items,'quality_items':quality_items,'weight_items':weight_items,'broken_items':broken_items,'wrong_items':wrong_items,'shortfall_items':shortfall_items,
        'packaging_charges':packaging_charges_after_gst}
    b2b_metrics = get_b2b_metrics(n_slot_id)
    resp.update(b2b_metrics)
    return resp


In [37]:
print(orders)

(13998,)


In [21]:
d=get_consumer_order_info('220823')

length of orders 13998
coupon_cost 11660.0
coupon_cost_with_cashback 11660.0
get_wallet_cost 0
referral_cost 7188.0
new_consumers 1251
new_agents 2
shortfall_refund 17185.00
sales 2561454.0
free_gifts 29240
issue_weight 104298.00
packagin_charges 57309.0
no_of_agents 527
no_of_mkt_agents 0
no_of_del_agents 527
num_of_deli_and_mkt_orders  13998
num_of_mkt_orders  13165
orders_count 13998
total_consumers 2136336
total_consumers 786861
total_consumers 4880
repeat_consumer 12747
repeat_reseller 525
repeat_reseller_per 99
total_active_resellers_per 9
weekly_active_consumers 54696
weekly_active_resellers 550
monthly_active_consumers 116547
monthly_active_resellers 597
shortfall_items 358
monthly_active_consumer_per 14
monthly_active_reseller_per 12
welcome_bonus 171025.00
scheme_bonus 11970.00
welcome_bonus used 19275.00
marketing_cost 317621
get_delivery_by_10am_per  94.57708072662354
2


In [10]:
d

{'orders_count': 13998,
 'no_of_items': 76908,
 'marketing_cost': 317621,
 'sales': 2460468.0,
 'no_of_agents': 527,
 'delivery_marketing_order': 833,
 'marketing_order': 13165,
 'no_of_mkt_agents': 0,
 'no_of_del_agents': 527,
 'avg_order_value': 175.77282468924133,
 'avg_order_value_per_agent': 4668.819734345351,
 'avg_item_value': 31,
 'avg_order_per_agent': 26.56,
 'item_per_order': 5.49,
 'repeat_consumers': 12747,
 'repeat_consumer_per': 91,
 'total_consumers': 2136336,
 'total_resellers': 51945,
 'weekly_active_consumers': 54696,
 'weekly_active_resellers': 550,
 'weekly_active_consumer_per': 6,
 'weekly_active_reseller_per': 11,
 'monthly_active_consumers': 116547,
 'monthly_active_resellers': 597,
 'monthly_active_reseller_per': 12,
 'monthly_active_consumer_per': 14,
 'referral_cost': 7188.0,
 'coupon_cost': 11660.0,
 'wallet_cost': 0,
 'wallet_balance': '27391033.19',
 'consumers': 554504,
 'avg_wallet': '49.40',
 'wallet_refund': 2413717.75,
 'refund_per': 98.1,
 'agent_com

In [16]:
val=[]
for i in range(len(list(d.values()))):
    if(list(d.values())[i]=='    '):
        val.append(0)
    else:
        val.append(int(float((list(d.values())[i]))))

In [23]:
len(val)

94

In [143]:
name=[]
for i in range(len(list(d.keys()))):
    name.append(list(d.keys())[i])

In [201]:
name

['orders_count',
 'no_of_items',
 'marketing_cost',
 'sales',
 'no_of_agents',
 'delivery_marketing_order',
 'marketing_order',
 'no_of_mkt_agents',
 'no_of_del_agents',
 'avg_order_value',
 'avg_order_value_per_agent',
 'avg_item_value',
 'avg_order_per_agent',
 'item_per_order',
 'repeat_consumers',
 'repeat_consumer_per',
 'total_consumers',
 'total_resellers',
 'weekly_active_consumers',
 'weekly_active_resellers',
 'weekly_active_consumer_per',
 'weekly_active_reseller_per',
 'monthly_active_consumers',
 'monthly_active_resellers',
 'monthly_active_reseller_per',
 'monthly_active_consumer_per',
 'referral_cost',
 'coupon_cost',
 'wallet_cost',
 'wallet_balance',
 'consumers',
 'avg_wallet',
 'wallet_refund',
 'refund_per',
 'agent_commission',
 'agent_commission_per',
 'five_stars',
 'four_stars',
 'lt_four_stars',
 'awaited',
 'total_cost',
 'total_cost_per',
 'five_stars_per',
 'four_stars_per',
 'awaited_per',
 'lt_four_stars_per',
 'active_consumers',
 'active_consumers_per',


'o'

In [182]:
def remove(x):
    return x.replace("'", "")

In [184]:
new_string=name[0].lstrip('\"')
new_string

'orders_count'

In [76]:
from collections import Counter

Counter([type(value) for value in m])

Counter({int: 68, float: 19, str: 7})

In [29]:
# k=tuple(k)

In [202]:
a=('orders_count', 'no_of_items', 'marketing_cost', 'sales', 'no_of_agents', 'delivery_marketing_order', 'marketing_order', 'no_of_mkt_agents', 'no_of_del_agents', 'avg_order_value', 'avg_order_value_per_agent', 'avg_item_value', 'avg_order_per_agent', 'item_per_order', 'repeat_consumers', 'repeat_consumer_per', 'total_consumers', 'total_resellers', 'weekly_active_consumers', 'weekly_active_resellers', 'weekly_active_consumer_per', 'weekly_active_reseller_per', 'monthly_active_consumers', 'monthly_active_resellers', 'monthly_active_reseller_per', 'monthly_active_consumer_per', 'referral_cost', 'coupon_cost', 'wallet_cost', 'wallet_balance', 'consumers', 'avg_wallet', 'wallet_refund', 'refund_per', 'agent_commission', 'agent_commission_per', 'five_stars', 'four_stars', 'lt_four_stars', 'awaited', 'total_cost', 'total_cost_per', 'five_stars_per', 'four_stars_per', 'awaited_per', 'lt_four_stars_per', 'active_consumers', 'active_consumers_per', 'total_active_resellers', 'total_active_resellers_per, shortfall_refund, shortfall_refund_per, nps, pre_del_nps, net_refund, net_refund_per, gateway_charges, new_consumers, blank, new_reseller, repeat_reseller, repeat_reseller_per, revenue_per_reseller, booked_value, weight_refund, dump_stock, shrinkage, total_wastage, total_wastage_per, logistics_cost, logistics_cost_per, gateway_charges_per, welcome_bonus, scheme_bonus, used_welcome_bonus, free_gift, delivery_10am_per, missing_items, quality_items, weight_items, broken_items, wrong_items, shortfall_items, packaging_charges, no_of_retailers, b2b_revenue, margin_b2b, logistics_b2b, logistics_b2b_per, net_margin_b2b, net_margin_b2b_per, available_stock, stock_sold, stock_sold_per)

SyntaxError: unterminated string literal (detected at line 1) (920941792.py, line 1)

In [37]:
q=("Create table fact.otipy_fact_matrix as Select * from (values "+str(tuple(val))+")AS t (daily_active_consumers, new_consumers , repeat_consumers , repeat_consumers_per , invited_consumers , transacted_consumers , transacted_consumers_per , weekly_active_consumers , weekly_active_consumers_per , monthly_active_consumers , monthly_active_consumers_per , daily_active_resellers , delivery_resellers, marketing_resellers, new_resellers , repeat_resellers , repeat_resellers_per , orders_per_reseller , revenue_per_reseller , onboarded_resellers , transacted_resellers , transacted_resellers_per , weekly_active_resellers , weekly_active_resellers_per , monthly_active_resellers , monthly_active_resellers_per , orders , delivery_marketing_order , marketing_order , aov , item_per_order , revenue_per_item , booked_value , shortfall_refund_value , weight_refund_value , packaging_charges , revenue , margin , margin_per , dump_stock , shrinkage , total_wastage , total_wastage_per , wallet_refund , wallet_refund_per , logistics_cost , logistics_cost_per , gateway_cost , gateway_cost_per , reseller_fee , reseller_fee_per , net_margin , net_margin_per , referrals , coupons , wallet_promo , welcome_bonus , scheme_bonus , used_welcome_bonus , free_gift , total_cost , total_cost_per , post_deliver_nps , pre_delivery_nps , delivery_per_by_10am , missing_items , quality_items , broken_items , weight_items , wrong_items , shortfall_items , five_stars , four_stars , less_than_4stars , awaited , five_stars_per , four_stars_per , less_than_4stars_per , awaited_per , wallet_balance , consumers , avg_wallet_balnce , no_of_retailers, b2b_revenue, margin_b2b, logistics_b2b, logistics_b2b_per, net_margin_b2b, net_margin_b2b_per, available_stock, stock_sold, stock_sold_per, n_slot_id , marketing_cost )")
cur.execute(q)
print(cur.fetchall())

[[1]]


In [24]:
d

{'orders_count': 13998,
 'no_of_items': 76908,
 'marketing_cost': 317621,
 'sales': 2460468.0,
 'no_of_agents': 527,
 'delivery_marketing_order': 833,
 'marketing_order': 13165,
 'no_of_mkt_agents': 0,
 'no_of_del_agents': 527,
 'avg_order_value': 175.77282468924133,
 'avg_order_value_per_agent': 4668.819734345351,
 'avg_item_value': 31,
 'avg_order_per_agent': 26.56,
 'item_per_order': 5.49,
 'repeat_consumers': 12747,
 'repeat_consumer_per': 91,
 'total_consumers': 2136336,
 'total_resellers': 51945,
 'weekly_active_consumers': 54696,
 'weekly_active_resellers': 550,
 'weekly_active_consumer_per': 6,
 'weekly_active_reseller_per': 11,
 'monthly_active_consumers': 116547,
 'monthly_active_resellers': 597,
 'monthly_active_reseller_per': 12,
 'monthly_active_consumer_per': 14,
 'referral_cost': 7188.0,
 'coupon_cost': 11660.0,
 'wallet_cost': 0,
 'wallet_balance': '27391033.19',
 'consumers': 554504,
 'avg_wallet': '49.40',
 'wallet_refund': 2413717.75,
 'refund_per': 98.1,
 'agent_com

In [22]:
daily_active_consumers=d['orders_count'],
new_consumers = d['new_consumers'],
repeat_consumers = d['repeat_consumers'],
repeat_consumers_per = d['repeat_consumer_per'],
invited_consumers = d['total_consumers'],
transacted_consumers = d['active_consumers'],
transacted_consumers_per = d['active_consumers_per'],
weekly_active_consumers = d['weekly_active_consumers'],
weekly_active_consumers_per = d['weekly_active_consumer_per'],
monthly_active_consumers = d['monthly_active_consumers'],
monthly_active_consumers_per = d['monthly_active_consumer_per'],
daily_active_resellers = d['no_of_agents'],
delivery_resellers=d['no_of_del_agents'],
marketing_resellers=d['no_of_mkt_agents'],
new_resellers = d['new_reseller'],
repeat_resellers = d['repeat_reseller'],
repeat_resellers_per = d['repeat_reseller_per'],
orders_per_reseller = d['avg_order_per_agent'],
revenue_per_reseller = d['revenue_per_reseller'],
onboarded_resellers = d['total_resellers'],
transacted_resellers = d['total_active_resellers'],
transacted_resellers_per = d['total_active_resellers_per'],
weekly_active_resellers = d['weekly_active_resellers'],
weekly_active_resellers_per = d['weekly_active_reseller_per'],
monthly_active_resellers = d['monthly_active_resellers'],
monthly_active_resellers_per = d['monthly_active_reseller_per'],
orders = d['orders_count'],
delivery_marketing_order = d['delivery_marketing_order'],
marketing_order = d['marketing_order'],
aov = d['avg_order_value'],
item_per_order = d['item_per_order'],
revenue_per_item = d['avg_item_value'],
booked_value = d['booked_value'],
shortfall_refund_value = d['shortfall_refund'],
weight_refund_value = d['weight_refund'],
packaging_charges = d['packaging_charges'],
revenue = d['sales'],
margin = d['margin_b2b'],
margin_per = 0
dump_stock = d['dump_stock'],
shrinkage = d['shrinkage'],
total_wastage = d['total_wastage'],
total_wastage_per = d['total_wastage_per'],
wallet_refund = d['wallet_refund'],
wallet_refund_per = d['refund_per'],
logistics_cost = d['logistics_cost'],
logistics_cost_per = d['logistics_cost_per'],
gateway_cost = d['gateway_charges'],
gateway_cost_per = d['gateway_charges_per'],
reseller_fee = d['agent_commission'],
reseller_fee_per = d['agent_commission_per'],
net_margin = 0,
net_margin_per =0,
referrals = d['referral_cost'],
coupons = d['coupon_cost'],
wallet_promo = d['wallet_cost'],   
welcome_bonus = d['welcome_bonus'],  
scheme_bonus = d['scheme_bonus'],
used_welcome_bonus = d['used_welcome_bonus'],
free_gift = d['free_gift'],
total_cost = d['total_cost'],
total_cost_per = d['total_cost_per'],
post_deliver_nps = d['nps'],
pre_delivery_nps = d['pre_del_nps'],
delivery_per_by_10am = d['delivery_10am_per'],
missing_items = d['missing_items'],
quality_items = d['quality_items'],
broken_items = d['broken_items'],
weight_items = d['weight_items'],
wrong_items = d['wrong_items'],
shortfall_items = d['shortfall_items'],
five_stars = d['five_stars'],
four_stars = d['four_stars'],
less_than_4stars = d['lt_four_stars'],
awaited = d['awaited'],
five_stars_per = d['awaited_per'],
four_stars_per = d['four_stars_per'],
less_than_4stars_per = d['lt_four_stars_per'],
awaited_per = d['awaited_per'],
wallet_balance = d['wallet_balance'],
consumers = d['consumers'],
avg_wallet_balnce = d['avg_wallet'],
no_of_retailers=d['no_of_retailers'],
b2b_revenue=d['b2b_revenue'],
margin_b2b=d['margin_b2b'],
logistics_b2b=d['logistics_b2b'],
logistics_b2b_per=d['logistics_b2b_per'],
net_margin_b2b=d['net_margin_b2b'],
net_margin_b2b_per=d['net_margin_b2b_per'],
available_stock=d['available_stock'],
stock_sold=d['stock_sold'],
stock_sold_per=d['stock_sold_per'],
# n_slot_id = n_slot_id,
marketing_cost = d['marketing_cost']


In [28]:
n_slot_id=220810

In [23]:
print("daily_active_consumers",daily_active_consumers),
print("new_consumers",new_consumers ), 
print("repeat_consumers",repeat_consumers ),
print("repeat_consumers_per ",repeat_consumers_per ),
print(" invited_consumers",invited_consumers ),
print("transacted_consumers ",transacted_consumers ),
print("transacted_consumers_per ",transacted_consumers_per ),
print("weekly_active_consumers ",weekly_active_consumers ),
print("weekly_active_consumers_per ",weekly_active_consumers_per ),
print("monthly_active_consumers ",monthly_active_consumers ),
print("monthly_active_consumers_per",monthly_active_consumers_per ),
print(" daily_active_resellers",daily_active_resellers ),
print(" delivery_resellers",delivery_resellers),
print("marketing_resellers ",marketing_resellers),
print("new_resellers",new_resellers ), 
print("repeat_resellers ",repeat_resellers ),
print(" repeat_resellers_per",repeat_resellers_per ), 
print(" orders_per_reseller",orders_per_reseller ), 
print("revenue_per_reseller ",revenue_per_reseller ), 
print("onboarded_resellers ",onboarded_resellers ), 
print("transacted_resellers ",transacted_resellers ), 
print(" transacted_resellers_per",transacted_resellers_per ), 
print("weekly_active_resellers ",weekly_active_resellers ), 
print(" weekly_active_resellers_per",weekly_active_resellers_per ), 
print(" monthly_active_resellers",monthly_active_resellers ),
print(" monthly_active_resellers_per",monthly_active_resellers_per ),
print("orders ",orders ),
print("delivery_marketing_order ",delivery_marketing_order ), 
print("marketing_order",marketing_order ),
print("aov ",aov ),
print("item_per_order ",item_per_order ), 
print("revenue_per_item ",revenue_per_item ), 
print("booked_value ",booked_value ), 
print("shortfall_refund_value ",shortfall_refund_value ), 
print("weight_refund_value ",weight_refund_value ),
print("packaging_charges ",packaging_charges ),
print("revenue ",revenue ), 
print("margin ",margin ), 
print(" margin_per",margin_per ), 
print(" dump_stock",dump_stock ),
print(" shrinkage",shrinkage ),
print("total_wastage ",total_wastage ),
print(" total_wastage_per",total_wastage_per ), 
print("wallet_refund ",wallet_refund ),
print("wallet_refund_per ",wallet_refund_per ), 
print("logistics_cost ",logistics_cost ), 
print(" logistics_cost_per",logistics_cost_per ), 
print("gateway_cost ",gateway_cost ),
print(" gateway_cost_per",gateway_cost_per ),
print(" reseller_fee",reseller_fee ), 
print("reseller_fee_per ",reseller_fee_per ), 
print("net_margin ",net_margin ),
print("net_margin_per ",net_margin_per ), 
print(" referrals",referrals ), 
print("coupons ",coupons ),
print("wallet_promo ",wallet_promo ),    
print(" welcome_bonus",welcome_bonus ),
print(" scheme_bonus",scheme_bonus ), 
print("used_welcome_bonus ",used_welcome_bonus ), 
print("free_gift ",free_gift ), 
print("total_cost ",total_cost ), 
print("total_cost_per ",total_cost_per ), 
print(" post_deliver_nps",post_deliver_nps ), 
print(" pre_delivery_nps",pre_delivery_nps ),
print("delivery_per_by_10am ",delivery_per_by_10am ),
print("missing_items ",missing_items ),
print(" quality_items",quality_items ), 
print(" broken_items",broken_items ), 
print("weight_items ",weight_items ), 
print(" wrong_items",wrong_items ), 
print("shortfall_items ",shortfall_items ), 
print(" five_stars",five_stars ), 
print(" four_stars",four_stars ), 
print(" less_than_4stars",less_than_4stars ),
print(" awaited",awaited ),
print("five_stars_per ",five_stars_per ), 
print("four_stars_per ",four_stars_per ),
print("less_than_4stars_per ",less_than_4stars_per ), 
print("awaited_per ",awaited_per ), 
print("wallet_balance ",wallet_balance ),
print("consumers ",consumers ),
print("avg_wallet_balnce ",avg_wallet_balnce ),
print("no_of_retailers ",no_of_retailers),
print("b2b_revenue ",b2b_revenue),
print("margin_b2b ",margin_b2b),
print("logistics_b2b ",logistics_b2b),
print("logistics_b2b_per ",logistics_b2b_per),
print("net_margin_b2b ",net_margin_b2b),
print("net_margin_b2b_per ",net_margin_b2b_per),
print("available_stock ",available_stock),
print("stock_sold ",stock_sold),
print("stock_sold_per ",stock_sold_per),
print("marketing_cost ",marketing_cost) 

daily_active_consumers (13998,)
new_consumers (1251,)
repeat_consumers (12747,)
repeat_consumers_per  (91,)
 invited_consumers (2136336,)
transacted_consumers  (786861,)
transacted_consumers_per  (36,)
weekly_active_consumers  (54696,)
weekly_active_consumers_per  (6,)
monthly_active_consumers  (116547,)
monthly_active_consumers_per (14,)
 daily_active_resellers (527,)
 delivery_resellers (527,)
marketing_resellers  (0,)
new_resellers (2,)
repeat_resellers  (525,)
 repeat_resellers_per (99,)
 orders_per_reseller (26.56,)
revenue_per_reseller  (4668,)
onboarded_resellers  (51945,)
transacted_resellers  (4880,)
 transacted_resellers_per (9,)
weekly_active_resellers  (550,)
 weekly_active_resellers_per (11,)
 monthly_active_resellers (597,)
 monthly_active_resellers_per (12,)
orders  (13998,)
delivery_marketing_order  (833,)
marketing_order (13165,)
aov  (175.77282468924133,)
item_per_order  (5.49,)
revenue_per_item  (31,)
booked_value  (2821766.0,)
shortfall_refund_value  ('17185.00',)
w